Попробуем решить следующую задачу: нам необходимо выгрузить сайт NPlus1.ru и положить статьи в базу данных. Отдельно должны лежать идентификаторы загруженных текстов, словари начальных форм и токенов, разбиение текста на предложения. Для решения задачи будем использовать базу данных MongoDB.

Импортируем все необходимые библиотеки. Обратите особое внимание на библиотеку для работы с Mongo - <a href="http://api.mongodb.com/python/current/api/index.html">pymongo</a> (еще одно описание <a href="https://pymongo.readthedocs.io/en/stable/">здесь</a>).

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import datetime
from tqdm import tqdm
import pymorphy2

import pymongo
from bson import ObjectId
from lxml import html
from pprint import pprint


Чтобы не ставить себе базу данных, можно использовать бесплатное облако <a href ="https://cloud.mongodb.com/">Mongo.Atlas</a>. Для этого необходимо зарегистрироваться, завести пользователя, создать проект, дать пользователю права на проект. Для пользователя получить строку для соединения с базой. 

Но в данном случае мы будем пользоваться MongoDB, запущенное на локальном компьютере при помощи Docker.

Для работы с БД используем класс MongoClient, в который передаем строку соединения.

In [2]:
# Version after 3.6
client = pymongo.MongoClient('localhost', 27017)
#client = pymongo.MongoClient()

Аналогом реляционных записей являются документы. Аналогом таблиц - коллекции и подколлекции. Сам документ может включать в себя другие документы. Документ записывается в формате JSON, но для Питона можно считать, что это словари.  
При помощи клиента обращаемся к базе `concordance`. Если эта база не существовала, она создастся при первой записи данных. Аналогично всё происходит с коллекциями.<br>

In [3]:
# Обращение к базе при помощи оператора квадратные скобки.
db = client['concordance']
# Обращение к базе при помощи оператора точка.
#db = client.concordance

Посмотрим что содержит в себе база.

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'concordance')

In [5]:
# Обращение к коллекции внутри базы при помощи оператора точка (но можно и при помощи квадратных скобок).
text_collection = db.nplus1texts
dictionary = db.dictionary
lemmas = db.lemmas
dbsents = db.sentences

vocabulary = db.vocabulary

В коллекции `nplus1texts` будем хранить адерса и идентификаторы текстов, в коллекции `dictionary` - словарь токенов, в `lemmas` - словарь лемм, наконец в `sentences` - разделение предложений на слова.

Для того, чтобы найти документ, необходимо использовать функцию `collection.find()`. Без параметров эта функция выдаст все документы.

In [8]:
for t in text_collection.find()[:2]:
    pprint(t)

{'_id': ObjectId('637f6f0d893f7efe906949d9'),
 'art_info': {'art_date': '20.01.20',
              'art_time': '14:18',
              'author': '\n'
                        '                                      \n'
                        '                  \n'
                        '                  Александр Войтюк\n'
                        '                ',
              'difficulty': '2.3'},
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к '
             'Красной планете этим летом, успешно прошел заключительные '
             'термовакуумные испытания, имитировавшие\xa0условия марсианской '
             'среды, в которых предстоит работать роверу,\xa0сообщается на '
             'сайте ESA.\n'
             'Старт второго этапа российско-европейской программы «ЭкзоМарс» '
             'намечен на период с 26 июля по 11 августа 2020 года. '
             'Ракета-носитель «Протон» выведет в космос перелетный модуль, '
             'который доставит к Марс

Посмотрим и на другие коллекции.

In [9]:
for t in dictionary.find()[:2]:
    pprint(t)

{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949db'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 1,
 'iniForm': 'марсоход',
 'token': 'Марсоход'}
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949de'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 2,
 'iniForm': 'розалинда',
 'token': 'Розалинд'}


In [10]:
for t in lemmas.find()[:2]:
    pprint(t)

{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949da'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 5,
 'iniForm': 'марсоход'}
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949dd'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 2,
 'iniForm': 'розалинда'}


In [11]:
for t in dbsents.find()[:2]:
    pprint(t)

{'_id': ObjectId('637f6f0d893f7efe906949dc'),
 'position': 1,
 'sent_id': 1,
 'textId': ObjectId('637f6f0d893f7efe906949d9'),
 'wordFormId': ObjectId('637f6f0d893f7efe906949db')}
{'_id': ObjectId('637f6f0d893f7efe906949df'),
 'position': 2,
 'sent_id': 1,
 'textId': ObjectId('637f6f0d893f7efe906949d9'),
 'wordFormId': ObjectId('637f6f0d893f7efe906949de')}


Чтобы поиск производился быстрее можно создать индексы.

In [12]:
# Добавляем индекс в коллекцию lemmas по полям iniForm и POS, оба по возрастанию.
# Заодно запрещаем добавлять повторяющиеся сочетания.
lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)
dictionary.create_index([('token', pymongo.ASCENDING), ('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

'token_1_iniForm_1_POS_1'

Или удалить их, если они больше не нужны.

In [7]:
# Для пробы чуть позже удалим индекс и посмотрим на изменение скорости работы.
lemmas.drop_index('iniForm_1_POS_1')

В качестве параметров функции передается документ (в случае Python - словарь) с полями и их значениями. 

Если значение должно отвечать не равенству, а другому условию, передается составное значение в виде словаря, ключем у которого будет обозначение оператора, а значением - значение с которым будет проводиться сравнение. Таких пар можно передать несколько.

In [13]:
# (freq _какое_ 2) _и_ (token _какое_ 'может')
#for l in dictionary.find({"freq": 2, 'token': 'может'}):
for l in dictionary.find({'token': 'ученые', 'freq': 16}):
    pprint(l)


{'POS': 'NOUN',
 '_id': ObjectId('637f6f0e893f7efe90694c6b'),
 'docs': [ObjectId('637f6f0e893f7efe90694c28'),
          ObjectId('637f6f10893f7efe90695273'),
          ObjectId('637f6f12893f7efe906956ff'),
          ObjectId('637f6f14893f7efe90695b7e'),
          ObjectId('637f6f1a893f7efe90696256'),
          ObjectId('637f6f1d893f7efe9069660c')],
 'freq': 16,
 'iniForm': 'учёный',
 'token': 'ученые'}


In [14]:
# freq _какое_ меньше _чего_ 5
for l in lemmas.find({"freq":{"$lt": 5}})[:5]:
    pprint(l)

print("----")
# freq _какое_ меньше _чего_ 5
for l in dictionary.find({"freq":{"$lt": 5}})[:5]:
    pprint(l)
    
print("----")
# freq _какое_ ((больше _чего_ 5) _и_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5, "$lt": 9}})[:5]:
    pprint(l)


{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949dd'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 2,
 'iniForm': 'розалинда'}
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949e0'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 2,
 'iniForm': 'франклин'}
{'POS': 'INFN',
 '_id': ObjectId('637f6f0d893f7efe906949e9'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 1,
 'iniForm': 'отправиться'}
{'POS': 'ADJF',
 '_id': ObjectId('637f6f0d893f7efe906949ef'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 3,
 'iniForm': 'красный'}
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949f8'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9'),
          ObjectId('637f6f21893f7efe906969af')],
 'freq': 2,
 'iniForm': 'лето'}
----
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949db'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 1,
 'iniForm': 'марсоход',
 'token': 'Марсоход'}
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d89

In [16]:
text_collection.find({"_id":ObjectId('637f6f0d893f7efe906949d9')})[0]

{'_id': ObjectId('637f6f0d893f7efe906949d9'),
 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes',
 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ',
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.\nСтарт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей мар

In [17]:
# Можно записать и так.
# (freq _какое_ (больше _чего_ 5)() _и_ (freq _какое_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5}, "freq": {"$lt": 9}})[:5]:
    pprint(l)

{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949db'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 1,
 'iniForm': 'марсоход',
 'token': 'Марсоход'}
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949de'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 2,
 'iniForm': 'розалинда',
 'token': 'Розалинд'}
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0d893f7efe906949e1'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 2,
 'iniForm': 'франклин',
 'token': 'Франклин'}
{'POS': 'ADJS',
 '_id': ObjectId('637f6f0d893f7efe906949e7'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9'),
          ObjectId('637f6f12893f7efe906956ff')],
 'freq': 3,
 'iniForm': 'должный',
 'token': 'должен'}
{'POS': 'INFN',
 '_id': ObjectId('637f6f0d893f7efe906949ea'),
 'docs': [ObjectId('637f6f0d893f7efe906949d9')],
 'freq': 1,
 'iniForm': 'отправиться',
 'token': 'отправиться'}


In [18]:
res1 = lemmas.find({"freq":{"$gt": 5}})
res1[0]['iniForm'], type(res1[0])

('который', dict)

Попробуем найти все документы со сложностью равной 2.3 (статья про ЭкзоМарс).

In [19]:
for l in text_collection.find({'art_info': {'difficulty': '6.8'}}):
    print(l)

In [21]:
text_collection.find({'art_info': {'art_date': '20.01.20',
  'art_time': '14:18',
  'difficulty': '2.3',
  'author': '\n                                      \n                  \n                  Александр Войтюк\n                '}})[0]

{'_id': ObjectId('637f6f0d893f7efe906949d9'),
 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes',
 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ',
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.\nСтарт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей мар

Ничего не найдено, так как такая запись предполагает полное совпадение, то есть поиск поддокумента с единственным полем и этим значением. Если полей будет несколько, роль будет играть даже порядок следования полей в базе!

То, что мы хотели сделать,нужно сделать вот так.

In [22]:
for l in text_collection.find({'art_info.difficulty': '2.3'}):
    pprint(l)

{'_id': ObjectId('637f6f0d893f7efe906949d9'),
 'art_info': {'art_date': '20.01.20',
              'art_time': '14:18',
              'author': '\n'
                        '                                      \n'
                        '                  \n'
                        '                  Александр Войтюк\n'
                        '                ',
              'difficulty': '2.3'},
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к '
             'Красной планете этим летом, успешно прошел заключительные '
             'термовакуумные испытания, имитировавшие\xa0условия марсианской '
             'среды, в которых предстоит работать роверу,\xa0сообщается на '
             'сайте ESA.\n'
             'Старт второго этапа российско-европейской программы «ЭкзоМарс» '
             'намечен на период с 26 июля по 11 августа 2020 года. '
             'Ракета-носитель «Протон» выведет в космос перелетный модуль, '
             'который доставит к Марс

Можно выбирать не все поля, а только те, которые нам необходимы.

In [23]:
for t in text_collection.find(projection=['art_info.difficulty','text_name'])[:5]:
    pprint(t)

{'_id': ObjectId('637f6f0d893f7efe906949d9'),
 'art_info': {'difficulty': '2.3'},
 'text_name': '\n'
              '            Марсоход миссии «ЭкзоМарс-2020» прошел '
              'термовакуумные испытания\n'
              '          '}
{'_id': ObjectId('637f6f0e893f7efe90694c28'),
 'art_info': {'difficulty': '4.2'},
 'text_name': '\n'
              '            Физики заперли свет в нанорезонаторе на рекордно '
              'долгое время\n'
              '          '}
{'_id': ObjectId('637f6f0f893f7efe90694ff3'),
 'art_info': {'difficulty': '1.7'},
 'text_name': '\n'
              '            Пилотируемый полет Crew Dragon к МКС состоится в '
              'первой половине 2020 года\n'
              '          '}
{'_id': ObjectId('637f6f10893f7efe90695273'),
 'art_info': {'difficulty': '3.9'},
 'text_name': '\n'
              '            Сон помог вознаграждению улучшить зрительное '
              'обучение\n'
              '          '}
{'_id': ObjectId('637f6f12893f7efe906956f

In [24]:
for t in text_collection.find(projection={'art_info.difficulty':True,'text_name':True, '_id':False})[:5]:
    pprint(t)

{'art_info': {'difficulty': '2.3'},
 'text_name': '\n'
              '            Марсоход миссии «ЭкзоМарс-2020» прошел '
              'термовакуумные испытания\n'
              '          '}
{'art_info': {'difficulty': '4.2'},
 'text_name': '\n'
              '            Физики заперли свет в нанорезонаторе на рекордно '
              'долгое время\n'
              '          '}
{'art_info': {'difficulty': '1.7'},
 'text_name': '\n'
              '            Пилотируемый полет Crew Dragon к МКС состоится в '
              'первой половине 2020 года\n'
              '          '}
{'art_info': {'difficulty': '3.9'},
 'text_name': '\n'
              '            Сон помог вознаграждению улучшить зрительное '
              'обучение\n'
              '          '}
{'art_info': {'difficulty': '1.9'},
 'text_name': '\n'
              '            Машинное обучение помогло распознать человека по '
              'танцу\n'
              '          '}


Ключевое слово `$or` и другие операторы позволяют применять логические свзки помимо логического И.

In [25]:
for d in dictionary.find({'$or':[{'token': 'объединяет'}, {'freq': 3}]})[:5]:
    print(d)

{'_id': ObjectId('637f6f0d893f7efe906949e7'), 'token': 'должен', 'iniForm': 'должный', 'POS': 'ADJS', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f12893f7efe906956ff')]}
{'_id': ObjectId('637f6f0d893f7efe906949f0'), 'token': 'Красной', 'iniForm': 'красный', 'POS': 'ADJF', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949fc'), 'token': 'успешно', 'iniForm': 'успешно', 'POS': 'ADVB', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f936de54ecc0e6e7dd22d')]}
{'_id': ObjectId('637f6f0d893f7efe90694a14'), 'token': 'среды', 'iniForm': 'среда', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28')]}
{'_id': ObjectId('637f6f0d893f7efe90694a2b'), 'token': 'сайте', 'iniForm': 'сайт', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f14893f7efe90695b7e')]}


Для поиска в массивах одного значения можно просто проводить поиск, БД сама будет искать объекты в массиве.

Если мы ищем несколько значений в массиве, необходимо использовать соответствующие предикаты: `$in` для поиска вхождения одного из элементов, `$all` - поиска всех элементов, и т.д.

Обратите внимание, для поиска мы используем не строку, а объект `ObjectId`, импортированный из библиотеки `bson`.

Помимо этого, во втором цикле в поле projection мы передаем не только какие поля надо выводить, но и какие поля выводить не надо. Так, поле \_id, хранящее идентификатор документа, по умолчанию должно выводиться. В примере ниже оно подавляется при помощи `"_id":False`.

In [26]:
for l in dictionary.find({'token': 'ученые'}):
    print(l['docs'][0])
    pprint(l)
    for d in dictionary.find({"docs": {"$in":[l['docs'][0]]}})[:5]:
        print(d)
    

637f6f0e893f7efe90694c28
{'POS': 'NOUN',
 '_id': ObjectId('637f6f0e893f7efe90694c6b'),
 'docs': [ObjectId('637f6f0e893f7efe90694c28'),
          ObjectId('637f6f10893f7efe90695273'),
          ObjectId('637f6f12893f7efe906956ff'),
          ObjectId('637f6f14893f7efe90695b7e'),
          ObjectId('637f6f1a893f7efe90696256'),
          ObjectId('637f6f1d893f7efe9069660c')],
 'freq': 16,
 'iniForm': 'учёный',
 'token': 'ученые'}
{'_id': ObjectId('637f6f0d893f7efe906949ed'), 'token': 'к', 'iniForm': 'к', 'POS': 'PREP', 'freq': 28, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f18893f7efe9069601c'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c'), ObjectId('637f6f21893f7efe906969af'), ObjectId('6384753a7b6d0cfe7a93a34c')]}
{'_id': ObjectId('637f6f0d893f7efe90694a14'), 'token': 'среды', 'iniForm': '

In [31]:
for d in dictionary.find({"docs": ObjectId('637f6f0e893f7efe90694c28')})[:3]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$in": [ObjectId('637f6f0d893f7efe906949d9')]}})[:3]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$all": [ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f0d893f7efe906949d9')]}},
                         projection={"_id":False, "token":True, "docs":True})[:3]:
    pprint(d)    

{'_id': ObjectId('637f6f0d893f7efe906949ed'), 'token': 'к', 'iniForm': 'к', 'POS': 'PREP', 'freq': 28, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f18893f7efe9069601c'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c'), ObjectId('637f6f21893f7efe906969af'), ObjectId('6384753a7b6d0cfe7a93a34c')]}
{'_id': ObjectId('637f6f0d893f7efe90694a14'), 'token': 'среды', 'iniForm': 'среда', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28')]}
{'_id': ObjectId('637f6f0d893f7efe90694a17'), 'token': 'в', 'iniForm': 'в', 'POS': 'PREP', 'freq': 215, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f12893f7efe906956ff'), ObjectId

In [32]:
# А это мы просто смотрим сколько текстов добавили в базу.
# В качестве параметра передается условие, считаются только удовлетворяющие ему документы.
# В данном случае нам нужны все.
print(text_collection.count_documents({}))

# Ищем количество слов с частотой встречаемости от 6 до 9.
print(lemmas.count_documents({"freq":{"$gt": 5, "$lt": 10}}))

# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим не выводить идентификатор и отсортировать всё по частоте.
for l in lemmas.find({"freq":{"$gt": 5, "$lt": 10}}, {"_id":False}).sort("iniForm")[:5]:
    print(l)
print("----")
# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим вывести токен и частоту, но не выводить идентификатор и отсортировать всё по частоте.
for l in dictionary.find({"freq":{"$gt": 5}}, {"token": True, "freq":True, "_id":False}, 
                         skip=25, limit=5).sort("freq"):
    print(l)
    
print("----")
for l in dictionary.find({"freq":{"$gt": 5}}, {"token": True, "freq":True, "_id":False}).sort("freq")[25:30]:
    print(l)


15
80
{'iniForm': 'алгоритм', 'POS': 'NOUN', 'freq': 7, 'docs': [ObjectId('637f6f12893f7efe906956ff'), ObjectId('637f6f16893f7efe90695e45'), ObjectId('637f6f18893f7efe9069601c')]}
{'iniForm': 'аппарат', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f13893f7efe90695970')]}
{'iniForm': 'атмосфера', 'POS': 'NOUN', 'freq': 8, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c')]}
{'iniForm': 'без', 'POS': 'PREP', 'freq': 6, 'docs': [ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f16893f7efe90695e45'), ObjectId('637f6f18893f7efe9069601c'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('6384753a7b6d0cfe7a93a34c')]}
{'iniForm': 'больший', 'POS': 'ADJF', 'freq': 8, 'docs': [ObjectId('637f6f14893f7efe90695b7e'), ObjectId('637f6f16893f7efe90695e45'), ObjectId('637f6f18893f7efe9069601c'), ObjectId('637f6f1

In [33]:
for d in dictionary.find({"docs": {"$size": 1}}, limit=5):
    print(d)


{'_id': ObjectId('637f6f0d893f7efe906949db'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949de'), 'token': 'Розалинд', 'iniForm': 'розалинда', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949e1'), 'token': 'Франклин', 'iniForm': 'франклин', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949ea'), 'token': 'отправиться', 'iniForm': 'отправиться', 'POS': 'INFN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949f0'), 'token': 'Красной', 'iniForm': 'красный', 'POS': 'ADJF', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}


Теперь напишем функцию для того, чтобы класть статью в БД. используем для этого функции:
- insert_one (insert, insert_many) - добавляет запись (несколько записей) в выбранную коллекцию.
- find_one_and_update - найти и обновить (аналог update).

В качестве параметра во все функции добавления передается документ (словарь), ключи которого содержат названия полей, а значения - значения этих полей. Значение поля само может являться документом. При поиске часть полей может принимать особые значения, например, `$ne` - не равно, `$size` - смотрим на размер массива, `$inc` - увеличить значение поля на заданную величину и т.д. Более подробно с соответствующими ключами можно ознакомиться <a href="https://metanit.com/nosql/mongodb/" >здесь</a>.

In [35]:
# Создаем морфоанализатор.
morph = pymorphy2.MorphAnalyzer()

In [36]:
class NPlus1Article:
    def __init__(self):
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        self.head=""
        self.text=""
        
    def __repr__(self):
        s = f"{self.head}\n{self.date} : {self.time} : {self.rubr} : {self.diff}\n"
        s += f"{self.author}\n{self.text}"
        return s
    
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "rubrics":"'+self.rubr+ \
            '", "difficulty":"'+self.diff+'", "title":"'+self.head+'", "author":"'+ \
            self.author+'","text":"'+self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "difficulty":self.diff,\
             "title":self.head, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res

def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    art = NPlus1Article()
    tree = html.fromstring(r.text)

    art.head = tree.xpath(".//h1")[0].text_content().strip()
    art.author = tree.xpath(".//span[contains(@class, 'whitespace-nowrap')]")[0].text_content().strip()
    art.time = tree.xpath(".//span[contains(@class, 'duration-75')]")[0].text_content()
    art.date = tree.xpath(".//span[contains(@class, 'duration-75')]")[1].text_content()
    art.diff = tree.xpath(".//span[contains(@class, 'duration-75')]")[2].text_content()
    art.rubr = tree.xpath(".//span[contains(@class, 'duration-75')]")[3].text_content()
    art.text = '\n'.join([p.text_content() for p in 
                 tree.xpath(".//div[contains(@class, 'mb-14')]//p[contains(@class, 'mb-6')]")]
                        )
    return art

def putNPlus1ArticleInMongo(art_url):
    # Загружаем текст статьи и другие ее части.
    art = getArticleTextNPlus1(art_url) 

    # Добавляем запись с информацией о статье.
    # inserted_id позволяет сразу получить идентификатор записи, чтобы потом ссылаться на него там, где это необходимо.
    a_text = {"text_url": art_url, "text_name": art.head, "art_text": art.text, 
              "art_info": {"art_date": art.date, "art_time": art.time, 
                           "difficulty": art.diff, "author": art.author}}
    text_id = text_collection.insert_one(a_text).inserted_id
    print(text_id)

    # Выделяем предложения (просто по точке с пробелом или в конце строки!!!).
    sents = re.split("\.\s|\.$", art.text)
    sent_num = 1
    # Загружаем предложения в базу.
    for s in tqdm(sents):
        # Выделяем слова (просто как группы русских букв!!!).
        words = re.findall("([А-Яа-я]+(\-[А-Яа-я]+)?)", s)
        posit = 1
        # Загрузка слов из предложений.
        for w in words:
            wf = morph.parse(w[0])
            # Провели морфологический анализ и теперь добавляем документ с двумя полями: нач. форма и часть речи.
            # Сперва проверяем есть ли там уже такая запись.
            # Смотрим сколько таких слов нашлось. Если ни одного - надо добавлять.
            if lemmas.count_documents({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                lemma_id = lemmas.insert_one({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS, 
                                              "freq": 1, "docs":[text_id]}).inserted_id
            # А если такое слово уже было, то обновляем частоту встречаемости и в каком документе оно встретиось.
            else:
                inis = lemmas.find({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
                lemmas.find_one_and_update({"_id": lemma_id}, 
                                           {"$inc": {"freq": 1}, 
                                            "$addToSet": {"docs": text_id}})
                # Вот таким образом можно добавлять номер документа столько раз, сколько в нем встретилось слово.
                #lemmas.find_one_and_update({"_id": lemma_id}, {"$push": {"docs": text_id}}) 

            # Повторяем операцию для токенов.
            if dictionary.count_documents({"token":w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                wf_id = dictionary.insert_one({"token":w[0], 
                                               "iniForm": wf[0].normal_form, 
                                               "POS": wf[0].tag.POS, 
                                               "freq": 1, 
                                               "docs": [text_id]}).inserted_id
            else:
                wrdf = dictionary.find({"token": w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                wf_id=wrdf[0]["_id"]
                dictionary.find_one_and_update({"_id": wf_id}, {"$inc":  {"freq": 1}}) 
                dictionary.find_one_and_update({"_id": wf_id}, {"$addToSet": {"docs": text_id}}) 
  


            if vocabulary.count_documents({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                lemma_id = vocabulary.insert_one({"iniForm": wf[0].normal_form, 
                                                  "POS": wf[0].tag.POS, 
                                                  "freq": 1, 
                                                  "tokens": [{"token": w[0].lower(), "freq": 1, 
                                                              "docs": [text_id]}]}
                                                ).inserted_id
            else:
                inis = vocabulary.find({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
                vocabulary.find_one_and_update({"_id": lemma_id}, 
                                               {"$inc": {"freq": 1}})#, 
#                                                 "$addToSet": {"token.docs": text_id}})

                if vocabulary.count_documents({"_id": lemma_id,
                                               "tokens.$.token": w[0].lower()
                                              }
                                             ) == 0:
                    _ = vocabulary.find_one_and_update({"_id": lemma_id}, 
                                                       {"$addToSet": {"tokens": {"token": w[0].lower(), "freq": 1, 
                                                        "docs": [text_id]}}}
                                                      )
                else:
                    _ = vocabulary.find_one_and_update({"_id": lemma_id, "tokens.token": w[0].lower()},
                                                       {"$inc": {"tokens.freq": 1}}
                                                      )


            # Добавляем номер предложения, идентификатор словоформы из словаря, позицию слова, из какого оно текста.
            dbsents.insert_one({"sent_id": sent_num, "wordFormId":wf_id, "position": posit, "textId": text_id})
            posit += 1  
        sent_num += 1

In [38]:
# Добавляем статью  в базу.
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/subwavelength-resonators")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/crew-dragon-first-crewed-flight")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/visual-perception")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/dancing-recognition")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/27/juno-pictures")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/04/12/alignment-of-jets")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/29/Ultra-Deep-Survey")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/balance")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/abel-prize-2018")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/10/12/srb-electrode")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/02/19/friction-on-ice")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/04/07/illex-coindetii")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/12/15/parachute-exomars-2022")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/26/flw")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/26/cure-rare")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/26/cheap-blaster-sound")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/01/27/cosmology-with-one-galaxy")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2022/02/04/keavision")
putNPlus1ArticleInMongo("https://nplus1.ru/material/2023/01/23/riddles-of-the-sphinx-AI")

63d250d2035df02b131d9efb


100%|███████████████████████████████████████████| 13/13 [00:00<00:00, 27.02it/s]


In [39]:
# for l in vocabulary.find({"docs": ObjectId('61fe63063bee6a4741b19f90')}):
for l in dictionary.find({"iniForm": 'американский'}):
# for l in vocabulary.find({'freq':{'$gt':1}})[:5]:
    pprint(l)

{'POS': 'ADJF',
 '_id': ObjectId('637f6f0f893f7efe9069508d'),
 'docs': [ObjectId('637f6f0f893f7efe90694ff3')],
 'freq': 1,
 'iniForm': 'американский',
 'token': 'американских'}
{'POS': 'ADJF',
 '_id': ObjectId('637f6f10893f7efe90695274'),
 'docs': [ObjectId('637f6f10893f7efe90695273')],
 'freq': 1,
 'iniForm': 'американский',
 'token': 'Американские'}
{'POS': 'ADJF',
 '_id': ObjectId('637f6f1d893f7efe906965dd'),
 'docs': [ObjectId('637f6f1a893f7efe90696256')],
 'freq': 1,
 'iniForm': 'американский',
 'token': 'американские'}


In [45]:
for l in vocabulary.find({"_id": ObjectId('63d250d2035df02b131d9efe'), "tokens.token": 'тысячелетиями'}):#,
    pprint(l)

{'POS': 'NOUN',
 '_id': ObjectId('63d250d2035df02b131d9efe'),
 'freq': 1,
 'iniForm': 'тысячелетие',
 'tokens': [{'docs': [ObjectId('63d250d2035df02b131d9efb')],
             'freq': 1,
             'token': 'тысячелетиями'}]}


In [46]:
# for l in text_collection.find({"_id": ObjectId('61fe63063bee6a4741b19f90')}):
for l in vocabulary.find()[:5]:#,
#                                                        {"$inc": {"tokens.$.freq": 1}}
#                                                       )
    pprint(l)

{'POS': 'NOUN',
 '_id': ObjectId('63d250d2035df02b131d9efe'),
 'freq': 1,
 'iniForm': 'тысячелетие',
 'tokens': [{'docs': [ObjectId('63d250d2035df02b131d9efb')],
             'freq': 1,
             'token': 'тысячелетиями'}]}
{'POS': 'ADJF',
 '_id': ObjectId('63d250d2035df02b131d9f02'),
 'freq': 1,
 'iniForm': 'искусственный',
 'tokens': [{'docs': [ObjectId('63d250d2035df02b131d9efb')],
             'freq': 1,
             'token': 'искусственный'}]}
{'POS': 'NOUN',
 '_id': ObjectId('63d250d2035df02b131d9f06'),
 'freq': 1,
 'iniForm': 'интеллект',
 'tokens': [{'docs': [ObjectId('63d250d2035df02b131d9efb')],
             'freq': 1,
             'token': 'интеллект'}]}
{'POS': 'VERB',
 '_id': ObjectId('63d250d2035df02b131d9f0a'),
 'freq': 1,
 'iniForm': 'обучаться',
 'tokens': [{'docs': [ObjectId('63d250d2035df02b131d9efb')],
             'freq': 1,
             'token': 'обучался'}]}
{'POS': 'PREP',
 '_id': ObjectId('63d250d2035df02b131d9f0c'),
 'freq': 2,
 'iniForm': 'на',
 'tokens': 

In [48]:
# Вот так можно удалить всё в базе. Если написать условие, то не всё. 
# Экспериментировать не будем, да?
text_collection.delete_many({})
dictionary.delete_many({})
lemmas.delete_many({})
dbsents.delete_many({})
#for l in dbsents.find():
# print(l)


In [47]:
# Так можно посмотреть все записи, у которых в токене записан список ровно из пяти элементов.
#for l in dictionary.find({"docs":{"$size":5}})[:5]:
for l in dictionary.find({"docs":{"$size":4}})[:5]:
    print(l)
print("=====")
# Так можно посмотреть все записи, у которых есть поле docs.
# for l in lemmas.find({"docs":{"$exists":True}})[:5]:
for l in text_collection.find({"coord":{"$exists":True}})[:5]:
    print(l)

{'_id': ObjectId('637f6f0d893f7efe90694a93'), 'token': 'поверхность', 'iniForm': 'поверхность', 'POS': 'NOUN', 'freq': 7, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c')]}
{'_id': ObjectId('637f6f0d893f7efe90694aa3'), 'token': 'качестве', 'iniForm': 'качество', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f14893f7efe90695b7e'), ObjectId('637f6f1a893f7efe90696256')]}
{'_id': ObjectId('637f6f0d893f7efe90694aa9'), 'token': 'работы', 'iniForm': 'работа', 'POS': 'NOUN', 'freq': 5, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f15893f7efe90695d2a'), ObjectId('637f6f18893f7efe9069601c')]}
{'_id': ObjectId('637f6f0d893f7efe90694ae4'), 'token': 'бы', 'iniForm': 'бы', 'POS': 'PRCL', 'freq': 4, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637

При помощи ключевого слова `$regex` можно искать в базе строки, отвечающие регулярным выражениям.

In [48]:
# iniForm _какое_ описывается_регуляркой _какой_ ^и.+
for d in lemmas.find({'iniForm':{'$regex':'^и.+'}})[:5]:
    print(d)
print("====")    
for d in dictionary.find({'token':{'$regex':'^[А-Я].+'}})[:5]:
    print(d)

{'_id': ObjectId('637f6f19893f7efe90696178'), 'iniForm': 'ива', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f18893f7efe9069601c')]}
{'_id': ObjectId('637f6f11893f7efe906956fc'), 'iniForm': 'ивтушок', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f12893f7efe906956ff')]}
{'_id': ObjectId('63d250d2035df02b131d9f0e'), 'iniForm': 'игра', 'POS': 'NOUN', 'freq': 4, 'docs': [ObjectId('63d250d2035df02b131d9efb')]}
{'_id': ObjectId('637f6f10893f7efe906952c9'), 'iniForm': 'играть', 'POS': 'VERB', 'freq': 3, 'docs': [ObjectId('637f6f10893f7efe90695273'), ObjectId('63d250d2035df02b131d9efb')]}
{'_id': ObjectId('63d250d2035df02b131da046'), 'iniForm': 'игровой', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('63d250d2035df02b131d9efb')]}
====
{'_id': ObjectId('63d250d2035df02b131da0b8'), 'token': 'АО', 'iniForm': 'ао', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('63d250d2035df02b131d9efb')]}
{'_id': ObjectId('637f6f19893f7efe906961f6'), 'token': 'Абелевска

Обратите внимание, что при обновлении заменяетя весь документ целиком. Чтобы избежать этого следует использовать ключевое слово `$set`, которое показывает, что обновляется значение только одного поля.

Да! И Монго умеет искать по регулярным выражениям!

In [49]:
dictionary.insert_one({'token':'Ъфыва', 'found':'false'})

In [50]:
dictionary.find_one_and_update({'token':{'$regex':'^Ъ.+'}, 'found':{"$exists":True}}, {'$set': {'found':'false'}})
# Просто посмотрим на документы
for d in dictionary.find({'token':{'$regex':'^Ъ.+'}, 'found':{"$exists":True}}):
    pprint(d)

{'_id': ObjectId('63d251a3035df02b131da0c9'),
 'found': 'false',
 'token': 'Ъфыва'}


In [58]:
# Просто посмотрим на документы
for d in dictionary.find({'token':{'$regex':'^И.+'}, 'found':{"$exists":True}}):
    print(d)

In [52]:
# А теперь найдем первое попавшееся слово, начинающееся с заглавное И и добавим ему новое поле found
dictionary.find_one_and_update({'token':{'$regex':'^И.+'}, 'found':'false'}, {'$set': {'found':'true'}})

In [53]:
for r in dictionary.find({'token': 'Изначально'}):
    print(r)

{'_id': ObjectId('637f6f0f893f7efe906951b9'), 'token': 'Изначально', 'iniForm': 'изначально', 'POS': 'ADVB', 'freq': 1, 'docs': [ObjectId('637f6f0f893f7efe90694ff3')]}


Ничего не получилось, так как поле found отсутствует у всех записей. Попробуем теперь найти первую запись, у которой нет такого поля: 'found':{"$exists":False}} .

In [60]:
dictionary.find_one_and_update({'token':{'$regex':'^А.+'}, 'found':{"$exists":False}}, {'$set': {'found':'true'}})

{'_id': ObjectId('637f6f0e893f7efe90694d82'),
 'token': 'Австралии',
 'iniForm': 'австралия',
 'POS': 'NOUN',
 'freq': 2,
 'docs': [ObjectId('637f6f0e893f7efe90694c28'),
  ObjectId('637f936de54ecc0e6e7dd22d')]}

In [134]:
# Попробуем найти все документы с полем found.
for d in dictionary.find({'token':{'$regex':'^А.+'}, 'found':{"$exists":True}}):
    print(d)

{'_id': ObjectId('616fcefce3381e0ff188131d'), 'token': 'Абелевской', 'iniForm': 'абелёвский', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('616fcefce3381e0ff188130d')], 'found': 'true'}
{'_id': ObjectId('616fcefce3381e0ff1881349'), 'token': 'Абелевскую', 'iniForm': 'абелёвский', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('616fcefce3381e0ff188130d')], 'found': 'true'}
{'_id': ObjectId('616fcef5e3381e0ff18803c5'), 'token': 'Австралии', 'iniForm': 'австралия', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('616fcef5e3381e0ff188026b')], 'found': 'true'}


А теперь опробуем еще две функции. `update_many` обновляет сразу все документы, попадающие под фильтр. А для того, чтобы удалить какое-то поле можно использовать `{"$unset": {<имя поля>:""}`.

In [70]:
dictionary.update_many({'token':{'$regex':'^Б.+'}}, {'$unset': {'found':""}})

In [71]:
# Попробуем найти все документы с полем found.
for d in dictionary.find({'token':{'$regex':'^Б.+'}, 'found':{"$exists":True}}):
    print(d)

In [72]:
dictionary.update_many({'token':{'$regex':'^Б.+'}, 'found':{"$exists":False}}, {'$set': {'found':'true'}})

In [73]:
# Попробуем найти все документы с полем found.
for d in dictionary.find({'token':{'$regex':'^Б.+'}, 'found':{"$exists":True}}):
    print(d)

{'_id': ObjectId('637f6f1b893f7efe9069626e'), 'token': 'Бактерии', 'iniForm': 'бактерия', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f1a893f7efe90696256')], 'found': 'true'}
{'_id': ObjectId('637f6f1b893f7efe90696331'), 'token': 'Бао', 'iniForm': 'бао', 'POS': None, 'freq': 1, 'docs': [ObjectId('637f6f1a893f7efe90696256')], 'found': 'true'}
{'_id': ObjectId('637f6f0f893f7efe906951ac'), 'token': 'Бенкен', 'iniForm': 'бенкен', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f0f893f7efe90694ff3')], 'found': 'true'}
{'_id': ObjectId('637f6f12893f7efe906957f8'), 'token': 'Беше', 'iniForm': 'беш', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f12893f7efe906956ff')], 'found': 'true'}
{'_id': ObjectId('637f6f22893f7efe90696baa'), 'token': 'Благодаря', 'iniForm': 'благодаря', 'POS': 'PREP', 'freq': 3, 'docs': [ObjectId('637f6f21893f7efe906969af'), ObjectId('6384753a7b6d0cfe7a93a34c')], 'found': 'true'}
{'_id': ObjectId('637f6f16893f7efe90695dcb'), 'token': 'Большого', 'iniForm': 'б

#### Важное замечание

Вся структура нашей базы данных отдает реляционностью. Никто не мешал нам сделать документ, вложенный в словарь начальных форм и хранить в нем все формы данного слова. Тогда если что, не надо будет делать запросы, в которых надо делать пересечение двух коллекций.

`{"iniForm":"абелёвский", 'POS': 'ADJF', "forms":[{'token': 'Абелевской', "tags":"род ед муж"}, {'token': 'Абелевский', "tags":"им ед муж"}]}`

## Работа с географическими данными

MongoDB позволяет работать с географическими данными. Основой является [GEO JSON](https://docs.mongodb.com/manual/reference/geojson/), позволяющий обрабатывать разные виды геолокаций.


In [75]:
text_collection.find_one_and_update(
    {'_id': ObjectId('637f6f0d893f7efe906949d9')},
    {"$set":{'coord':{ 'type': "Point", 'coordinates': [ -20, -15 ] }}})


{'_id': ObjectId('637f6f0d893f7efe906949d9'),
 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes',
 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ',
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.\nСтарт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей мар

In [76]:
for a in text_collection.find({'_id': ObjectId('637f6f0d893f7efe906949d9')}):
    print(a)

{'_id': ObjectId('637f6f0d893f7efe906949d9'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.\nСтарт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсох

In [77]:
for d in text_collection.find({'coord':{'$geoIntersects':{"$geometry": 
{"type": "Polygon",
 "coordinates": [[[15,10], [15,20],[25,20], [25,10], [15,10]]]
}}}}):
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих п

#### Важное замечание

!!! Если не построить этот индекс, то искаться не будет!!!

In [78]:
#lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

text_collection.create_index([("coord", pymongo.GEOSPHERE)], unique=False)

'coord_2dsphere'

In [79]:
req=text_collection.find(
{
   "coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ 19.999 , 15.001 ]
       },
       "$maxDistance": 1000,
       "$minDistance": 0
     }
   }
}
)
#req
for d in req:    
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих п

In [80]:
for d in text_collection.find({'coord':{'$geoWithin':{"$box": 
[[15,10], [25,20]]
}}}):
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих п

## Полнотекстовый поиск

Для того, чтобы заработал полнотекстовый поиск, необходимо добавить полнотекстовый индекс по полям, в которых мы будем искать. После этого поиск будет проводиться по всем проиндексированным полям.

In [81]:
#text_collection.create_index([('text_name', pymongo.ASCENDING)], unique=False)
text_collection.create_index([('art_text', pymongo.TEXT)], unique=False)


'art_text_text'

In [149]:
#for d in text_collection.find({'text_name': 'Бактерии помогли получить катализатор для электролиза воды'}):
# for d in text_collection.find({
#   "$text":{"$search": "воды"}}):

print(text_collection.count_documents({ "$text": { "$search": "учеными" } } ))


#for d in text_collection.find({'art_text':{'$regex':'воды'}}):
for d in text_collection.find({ "$text": { "$search": "Американские учеными" } } ):
# for d in text_collection.find({ "$text": { "$search": "Американ" } } ):
    pos_amer = d["art_text"].find("американские")
    if pos_amer == -1:
        pos_amer = d["art_text"].find("Американские")
    pos_sci = d["art_text"].replace("\n", " ").find("учеными")
    if pos_sci == -1:
        pos_sci = d["art_text"].replace("\n", " ").find("Учеными")
    print(d["text_name"], f'\n[...{d["art_text"][max(pos_amer-20, 0): pos_amer+20]}..., ...{d["art_text"][pos_sci-20: pos_sci+20]}...]')
    pos_amer = d["art_text"].find("американск")
    if pos_amer == -1:
        pos_amer = d["art_text"].find("Американск")
    pos_sci = d["art_text"].replace("\n", " ").find(" учен")
    if pos_sci == -1:
        pos_sci = d["art_text"].replace("\n", " ").find("Учен")
    print(f'[...{d["art_text"][max(pos_amer-20, 0): pos_amer+20]}..., ...{d["art_text"][pos_sci-20: pos_sci+20]}...]')

2

            Бактерии помогли получить катализатор для электролиза воды
           
[...бах.
В
прошлом году американские и китай..., ......]
[...бах.
В
прошлом году американские и китай..., ...цесс меньше энергии,
ученые покрывают по...]

            Сон помог вознаграждению улучшить зрительное обучение
           
[...Американские ученые ..., ......]
[...Американские ученые ..., ......]

            Астрономы внезапно нашли сонаправленные джеты
           
[...Астрономы из Кейпта..., ...табах было замечено учеными впервые и не...]
[...Астрономы из Кейпта..., ...штабах было замечено учеными впервые и н...]

            Физики заперли свет в нанорезонаторе на рекордно долгое время
           
[...Физики создали резо..., ...ения.
Разработанный учеными резонатор мо...]
[...Физики создали резо..., ...аний. На основе него ученые создали устр...]


In [124]:
for d in text_collection.find():
    print(d)

{'_id': ObjectId('637f6f0d893f7efe906949d9'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.\nСтарт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсох

А теперь посмотрим на группировку, ограничение выдачи и еще кое-что.

In [140]:
def search_any(collection, query, top_n, sort='date'):
    if sort == 'score':
        result = collection.find({'$text': {'$search': query}}, # Можно брать фильтр по записям, а потом в них искать.
                                 {'score': {'$meta': 'textScore'}}) # Можно брать скор от Монги.
        result.sort([('score', {'$meta': 'textScore'})]).limit(top_n)
    else:
        result = collection.find({'$text': {'$search': query}}).sort('date', pymongo.DESCENDING).limit(top_n)
    return result

def search_all(collection, query, top_n, sort='date'):
    if sort == 'score':
        result = collection.find({'$text': {'$search': f'\"{query}\"'}},
                                 {'score': {'$meta': 'textScore'}})
        result.sort([('score', {'$meta': 'textScore'})]).limit(top_n)
    else:
        result = collection.find({'$text': {'$search': f'\"{query}\"'}}).\
                            sort('date', pymongo.DESCENDING).limit(top_n)

    return result

def search_and_group(collection, query, find='any'):
    if find == 'all':
        query = f'\"{query}\"'

    toxic_count = collection.aggregate([{'$match': {'$text': {'$search': query}}},
                                        {'$group': {'_id': '$art_info.art_date',
                                                    'count': {'$sum': 1}}},
#                                         {'$project': {'_id': 0, 'is_toxic': '$_id',
#                                                       'count': 1, 'sum': 1}}
                                       ])
    return toxic_count

In [127]:
for d in search_any(text_collection, "учеными", 5, 'art_info.art_date'):
    pprint(d)

{'_id': ObjectId('637f6f14893f7efe90695b7e'),
 'art_info': {'art_date': '12.04.16',
              'art_time': '15:26',
              'author': '\n'
                        '                                      \n'
                        '                  \n'
                        '                  Кристина Уласович\n'
                        '                ',
              'difficulty': '3.3'},
 'art_text': 'Астрономы из\xa0Кейптаунского Университета и\xa0Университета '
             'Западно-Капской провинции в\xa0Южной Африке обнаружили массивные '
             'черные дыры, джеты которых ориентированы в\xa0одном направлении. '
             'Подобное явление на больших масштабах было замечено учеными '
             'впервые и не было \n'
             'предсказано существующими теориями, поэтому в будущем его '
             'только \n'
             'предстоит изучить. Работа авторов опубликована в\xa0журнале '
             'Monthly Notices of\xa0the Royal Astronomical Society, 

In [141]:
for d in search_and_group(text_collection, "в"):
    pprint(d)

{'_id': '28.11.22', 'count': 1}
{'_id': '18.01.20', 'count': 2}
{'_id': '27.06.16', 'count': 1}
{'_id': '24.11.22', 'count': 1}
{'_id': '20.03.18', 'count': 2}
{'_id': '12.04.16', 'count': 1}
{'_id': '19.02.21', 'count': 1}
{'_id': '20.01.20', 'count': 3}
{'_id': '5.7', 'count': 1}
{'_id': '12.10.20', 'count': 1}
{'_id': '07.04.21', 'count': 1}
{'_id': '29.06.16', 'count': 1}
